**Fine tune a VITS model with the Coqui TTS framework using audio samples of your choice.**

Thank you to all of the [Coqui TTS](https://https://github.com/coqui-ai/TTS) contributors, the OpenAI team,and [@Thorsten-Voice](https://https://www.youtube.com/c/ThorstenMueller ) (I think he wrote this original script). I have no idea who wrote the rnnoise loop, but they are great, too. I just smash things together. [-nn](https://https://www.youtube.com/c/NanoNomad )

If you have run the script before, and want to examine your training session using Tensorboard, set your dataset directories, then skip to the bottom section to load Tensorboard.

Run this cell to connect your Google Drive account to save files.

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


Set dataset and trainer output directory (model states saved here, subdirectory of dataset). Set upload_dir and place all audio files (Wav, mp3) in upload directory which will be /content/drive/MyDrive/{upload_dir}

In [ ]:
dataset_name = "james-dataset" #@param {type:"string"}
output_directory = "traineroutput" #@param {type:"string"}
upload_dir = "mp3uploads" #@param {type:"string"}
upload_dir = "/content/drive/MyDrive/" + upload_dir
!mkdir $upload_dir

Cleanup previous audio processing

In [ ]:
!rm -rf /content/drive/MyDrive/$upload_dir/out
!rm -rf /content/drive/MyDrive/$dataset_name/converted

Download and Build Rnnoise (https://github.com/xiph/rnnoise) and Requirements (Optional if not processing .wav files for a new dataset. Multiple passes of rnnoise over samples may degrade audio quality. You should also preview your output dataset before training to ensure that the processing has not degraded any of your samples.  Occasionally, it will denoise too much of the tail end of phrases)

In [ ]:
!pip install pyloudnorm
!git clone https://github.com/xiph/rnnoise.git
!sudo apt-get install curl autoconf automake libtool python-dev pkg-config sox ffmpeg
%cd /content/rnnoise
!sh autogen.sh
!sh configure
!make clean
!make

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'rnnoise'...
remote: Enumerating objects: 420, done.
remote: Counting objects: 100% (219/219), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 420 (delta 194), reused 192 (delta 192), pack-reused 201
Receiving objects: 100% (420/420), 836.04 KiB | 7.03 MiB/s, done.
Resolving deltas: 100% (221/221), done.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'python-dev-is-python2' instead of 'python-dev'
autoconf is already the newest version (2.69-11.1).
autoconf set to manually installed.
automake is already the newest version (1:1.16.1-4ubuntu6).
automake set to manually installed.
pkg-config is already the newest version (0.29.1-0ubuntu4).
curl is already the newest version (7.68.0-1ubuntu2.15).
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
The following package was automatically i

Install Sox, Install OpenAI Whisper STT+Translation (https://github.com/openai/whisper)

In [ ]:
%cd /content
!sudo apt install sox
!git clone https://github.com/openai/whisper.git
!pip install git+https://github.com/openai/whisper.git


/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
sox is already the newest version (14.4.2+git20190427-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Cloning into 'whisper'...
remote: Enumerating objects: 479, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 479 (delta 83), reused 111 (delta 69), pack-reused 343
Receiving objects: 100% (479/479), 7.41 MiB | 21.19 MiB/s, done.
Resolving deltas: 100% (285/285), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-b1bte6qd
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-b1bte6

Install Coqui TTS (https://github.com/coqui-ai/TTS), espeak-ng phonemeizer (https://github.com/espeak-ng/espeak-ng), download Coqui TTS source and examples from GitHub

In [ ]:
#@title
%cd /content
!sudo apt-get install espeak-ng
!git clone https://github.com/coqui-ai/TTS.git
!pip install TTS
!tts --list_models

/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 21 not upgraded.
Need to get 4,215 kB of archives.
After this operation, 12.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libpcaudio0 amd64 1.1-4 [7,908 B]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libsonic0 amd64 0.2.0-8 [13.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/main amd64 espeak-ng-data amd64 1.50+dfsg-6 [3,682 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/main amd64 libespeak-ng1 amd64 1.50+dfsg-6 [189 

Convert mp3 files in upload_dir to wav as 22050hz mono wav
Place output wav files in directory named 'out'.
Copy any uploaded .wav files to 'out' as 22050hz mono wav.

Make directory 'splits'. Use sox to split wav files into 8 second segments, placed in 'splits'.

In [ ]:
#@title
%cd $upload_dir
!rm -rf $upload_dir/out
!mkdir $upload_dir/out
!find . -name '*.mp3' -exec bash -c 'for f; do ffmpeg -i "$f" -acodec pcm_s16le -ar 22050 -ac 1 out/"${f%.mp3}".wav ; done' _ {} +
!find . -name '*.wav' -exec bash -c 'for f; do ffmpeg -i "$f" -acodec pcm_s16le -ar 22050 -ac 1 out/"${f%.wav}".wav ; done' _ {} +


!ls -al $upload_dir/out
!mkdir $upload_dir/out/splits
%cd $upload_dir/out
!for FILE in *.wav; do sox "$FILE" splits/"$FILE" --show-progress silence 1 0.5 0.1% 1 0.5 0.1% : newfile : restart ; done
#alt split method: force splits of 8 seconds, however this will split words. Comment the above with # and remove the # below to change
#!for FILE in *.wav; do sox "$FILE" splits/"$FILE" --show-progress trim 0 8 : restart ; done
%cd $upload_dir/out/splits
!find . -name "*.wav" -type f -size -15k -delete

Streaming output truncated to the last 5000 lines.
In:44.6% 00:04:00.00 [00:04:57.76] Out:4.98M [  ====|====  ]        Clip:0    
In:47.5% 00:04:15.61 [00:04:42.16] Out:5.30M [  -===|===-  ]        Clip:0    
In:51.1% 00:04:34.55 [00:04:23.21] Out:5.67M [  -===|===-  ]        Clip:0    
In:70.3% 00:06:17.84 [00:02:39.93] Out:7.94M [  ====|====  ]        Clip:0    
In:71.4% 00:06:23.78 [00:02:33.99] Out:8.05M [  -===|===-  ]        Clip:0    
In:72.7% 00:06:30.84 [00:02:26.93] Out:8.16M [  ====|====  ]        Clip:0    
In:74.1% 00:06:38.27 [00:02:19.50] Out:8.31M [  ====|====  ]        Clip:0    
In:76.7% 00:06:52.39 [00:02:05.38] Out:8.61M [  ====|====  ]        Clip:0    
In:79.5% 00:07:07.62 [00:01:50.15] Out:8.94M [  ====|====  ]        Clip:0    
In:83.5% 00:07:28.80 [00:01:28.97] Out:9.39M [  ====|====  ]        Clip:0    
In:85.1% 00:07:37.71 [00:01:20.05] Out:9.57M [  -===|===-  ]        Clip:0    
In:88.2% 00:07:54.06 [00:01:03.71] Out:9.91M [  -===|===-  ]        Clip:0    
I

(Optional) Run rnnoise, processed clips will be saved to your Google Drive in a folder named 'converted'. Files converted to 22khz mono for VITS model fine tuning.

Original author unknown, but thank you for posting this helpful code.

In [ ]:
#@title
orig_wavs= upload_dir + "/out/splits"
from pathlib import Path
import os
import subprocess
import soundfile as sf
import pyloudnorm as pyln
import sys
import glob
rnn = "/content/rnnoise/examples/rnnoise_demo"
paths = glob.glob(os.path.join(orig_wavs, '*.wav'))
for filepath in paths:
  base = os.path.basename(filepath)
  tp_s = "/content/drive/MyDrive/" + dataset_name + "/" + "converted" + "/"
  tf_s = "/content/drive/MyDrive/" + dataset_name + "/" + "converted" + "/" + base
  target_path = Path(tp_s)
  target_file = Path(tf_s)
  print("From: " + str(filepath))
  print("To: " + str(target_file))

# Stereo to Mono; upsample to 48000Hz
# added -G to fix gain, -v 0.95
  subprocess.run(["sox", "-G", "-v", "0.95", filepath, "48k.wav", "remix", "-", "rate", "48000"])
  subprocess.run(["sox", "48k.wav", "-c", "1", "-r", "48000", "-b", "16", "-e", "signed-integer", "-t", "raw", "temp.raw"]) # convert wav to raw
  subprocess.run(["/content/rnnoise/examples/rnnoise_demo", "temp.raw", "rnn.raw"]) # apply rnnoise
  subprocess.run(["sox", "-G", "-v", "0.95", "-r", "48k", "-b", "16", "-e", "signed-integer", "rnn.raw", "-t", "wav", "rnn.wav"]) # convert raw back to wav

  subprocess.run(["mkdir", "-p", str(target_path)])
  subprocess.run(["sox", "rnn.wav", str(target_file), "remix", "-", "highpass", "100", "lowpass", "7000", "rate", "22050"]) # apply high/low pass filter and change sr to 22050Hz
  data, rate = sf.read(target_file)

# peak normalize audio to -1 dB
  peak_normalized_audio = pyln.normalize.peak(data, -1.0)

# measure the loudness first
  meter = pyln.Meter(rate) # create BS.1770 meter
  loudness = meter.integrated_loudness(data)

# loudness normalize audio to -25 dB LUFS
  loudness_normalized_audio = pyln.normalize.loudness(data, loudness, -25.0)
  sf.write(target_file, data=loudness_normalized_audio, samplerate=22050)
  print("")
%cd /content/drive/MyDrive/"$dataset_name"/converted/
!find . -name "*.wav" -type f -size -15k -delete


Streaming output truncated to the last 5000 lines.
From: /content/drive/MyDrive/mp3uploads/out/splits/ispy_21_lincoln_64kb017.wav
To: /content/drive/MyDrive/james-dataset/converted/ispy_21_lincoln_64kb017.wav

From: /content/drive/MyDrive/mp3uploads/out/splits/ispy_21_lincoln_64kb018.wav
To: /content/drive/MyDrive/james-dataset/converted/ispy_21_lincoln_64kb018.wav

From: /content/drive/MyDrive/mp3uploads/out/splits/ispy_21_lincoln_64kb019.wav
To: /content/drive/MyDrive/james-dataset/converted/ispy_21_lincoln_64kb019.wav

From: /content/drive/MyDrive/mp3uploads/out/splits/ispy_21_lincoln_64kb020.wav
To: /content/drive/MyDrive/james-dataset/converted/ispy_21_lincoln_64kb020.wav

From: /content/drive/MyDrive/mp3uploads/out/splits/ispy_21_lincoln_64kb021.wav
To: /content/drive/MyDrive/james-dataset/converted/ispy_21_lincoln_64kb021.wav

From: /content/drive/MyDrive/mp3uploads/out/splits/ispy_21_lincoln_64kb022.wav
To: /content/drive/MyDrive/james-dataset/converted/ispy_21_lincoln_64kb022.

KeyboardInterrupt: ignored

Copy wav files to dataset wav directory

In [ ]:
#@title
!mkdir /content/drive/MyDrive/$dataset_name/wavs
!cp /content/drive/MyDrive/$dataset_name/converted/*.wav /content/drive/MyDrive/$dataset_name/wavs
!ls -al /content/drive/MyDrive/$dataset_name/wavs

total 4278656
-rw------- 1 root root   729458 Feb 10 11:41 catspaw_01_lincoln_64kb001.wav
-rw------- 1 root root  4767696 Feb 10 11:41 catspaw_01_lincoln_64kb002.wav
-rw------- 1 root root   109852 Feb 10 11:41 catspaw_01_lincoln_64kb003.wav
-rw------- 1 root root  3881284 Feb 10 11:41 catspaw_01_lincoln_64kb004.wav
-rw------- 1 root root  6029836 Feb 10 11:41 catspaw_01_lincoln_64kb005.wav
-rw------- 1 root root   312272 Feb 10 11:41 catspaw_02_lincoln_64kb001.wav
-rw------- 1 root root    36648 Feb 10 11:41 catspaw_02_lincoln_64kb002.wav
-rw------- 1 root root  5886070 Feb 10 11:41 catspaw_02_lincoln_64kb003.wav
-rw------- 1 root root  1849156 Feb 10 11:41 catspaw_02_lincoln_64kb004.wav
-rw------- 1 root root  1192068 Feb 10 11:41 catspaw_02_lincoln_64kb005.wav
-rw------- 1 root root  2948570 Feb 10 11:41 catspaw_02_lincoln_64kb006.wav
-rw------- 1 root root  1857094 Feb 10 11:41 catspaw_02_lincoln_64kb007.wav
-rw------- 1 root root   476324 Feb 10 11:41 catspaw_02_lincoln_64kb008.wa

Run Whisper on generated audio clips, generate transcript named metadata.csv in LJSpeech format in the dataset directory.

In [ ]:
#@title
import whisper
import os, os.path
import glob
import pandas as pd

from pathlib import Path

wavs = '/content/drive/MyDrive/'+dataset_name+'/wavs'

model = whisper.load_model("medium.en")
paths = glob.glob(os.path.join(wavs, '*.wav'))
print(len(paths))

all_filenames = []
transcript_text = []
with open('/content/drive/MyDrive/'+dataset_name+'/metadata.csv', 'w', encoding='utf-8') as outfile:
	for filepath in paths:
		base = os.path.basename(filepath)
		all_filenames.append(base)
	for filepath in paths:
		result = model.transcribe(filepath)
		output = result["text"].lstrip()
		output = output.replace("\n","")
		thefile = str(os.path.basename(filepath).lstrip(".")).rsplit(".")[0]
		outfile.write(thefile + '|' + output + '|' + output + '\n')
		print(thefile + '|' + output + '|' + output + '\n')


100%|██████████████████████████████████████| 1.42G/1.42G [00:11<00:00, 134MiB/s]


4959
christmashoneymoon_01_mathews_64kb001|one of a Christmas honeymoon.|one of a Christmas honeymoon.

christmashoneymoon_01_mathews_64kb002|Fuck doing."|Fuck doing."

christmashoneymoon_01_mathews_64kb003|Recording by Jen Broda|Recording by Jen Broda

christmashoneymoon_01_mathews_64kb004|Christmas Honeymoon by Frances Amar Matthews.|Christmas Honeymoon by Frances Amar Matthews.

christmashoneymoon_01_mathews_64kb005|Chapter 1|Chapter 1

christmashoneymoon_01_mathews_64kb006|Betty Revere wrote to Ani de Pastor.|Betty Revere wrote to Ani de Pastor.

christmashoneymoon_01_mathews_64kb007|Christmas Eve in the morning at Willard's Hotel, Washington, D.C.|Christmas Eve in the morning at Willard's Hotel, Washington, D.C.

christmashoneymoon_01_mathews_64kb008|My dearest girl, I am sitting up with ten pillows at my back. It is only six o'clock a.m., but I can't sleep another wink. Not that I have slept, for I haven't not a moment since I laid down at two a.m. four hours ago, back from the c

Display dataset

In [ ]:
#@title
!cat /content/drive/MyDrive/$dataset_name/metadata.csv

babysopera_01_crane_64kb001|Song number one of the baby's opera.|Song number one of the baby's opera.
babysopera_01_crane_64kb002|This is a LibriVox recording. All LibriVox recordings are in the public domain.|This is a LibriVox recording. All LibriVox recordings are in the public domain.
babysopera_01_crane_64kb003|For more information or to volunteer, please visit libriVox.org.|For more information or to volunteer, please visit libriVox.org.
babysopera_01_crane_64kb004|Recording by Jen Broda.|Recording by Jen Broda.
babysopera_01_crane_64kb005|Crane.|Crane.
babysopera_01_crane_64kb006|girls and boys.|girls and boys.
babysopera_01_crane_64kb007|Girls and boys come out to play The moor doves shine as bright as day Leave your summer and leave your sleep Come to your playfellows in the street Come with a whoop and come with a call Come with a good will or not at all Walk the ladder and down the wall A penny loaf will serve you all|Girls and boys come out to play The moor doves shine as b

Download VITS model and Generate Sample Wav File to /content/ljspeech-vits.wav  This will be deleted when your Colab session is closed.

In [ ]:
!tts --text "I am the very model of a modern Major General" --model_name "tts_models/en/ljspeech/vits" --out_path /content/ljspeech-vits.wav

 > Downloading model to /root/.local/share/tts/tts_models--en--ljspeech--vits
100% 146M/146M [00:28<00:00, 5.15MiB/s]
 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Text: I a

Save the training script with your dataset name, and output dir set. File will be saved in your Google drive dataset folder as train_vits.py

In [ ]:
#@title
code = """import os

from trainer import Trainer, TrainerArgs

from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import Vits, VitsAudioConfig
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

#output_path = os.path.dirname(os.path.abspath(__file__))
##########################################
#Change this to your dataset directory
##########################################
output_path = "/content/drive/MyDrive/"""
code = code + dataset_name + "/" + output_directory + "/" + "\""

code=code + """
dataset_config = BaseDatasetConfig(
##########################################
#Change this to your dataset directory
##########################################
    formatter="ljspeech", meta_file_train="metadata.csv", path=os.path.join(output_path, "/content/drive/MyDrive/"""
code = code + dataset_name
code=code + """")

)
audio_config = VitsAudioConfig(
    sample_rate=22050, win_length=1024, hop_length=256, num_mels=80, mel_fmin=0, mel_fmax=None
)

config = VitsConfig(
    audio=audio_config,
    run_name="vits_ljspeech",
    batch_size=12,
    eval_batch_size=12,
    batch_group_size=4,
#    num_loader_workers=8,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100000,
    save_step=1000,
	  save_checkpoints=True,
	  save_n_checkpoints=4,
	  save_best_after=1000,
    #text_cleaner="english_cleaners",
    text_cleaner="multilingual_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=True,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    cudnn_benchmark=False,
)

# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# config is updated with the default characters if not defined in the config.
tokenizer, config = TTSTokenizer.init_from_config(config)

# LOAD DATA SAMPLES
# Each sample is a list of ```[text, audio_file_path, speaker_name]```
# You can define your custom sample loader returning the list of samples.
# Or define your custom formatter and pass it to the `load_tts_samples`.
# Check `TTS.tts.datasets.load_tts_samples` for more details.
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

# init model
model = Vits(config, ap, tokenizer, speaker_manager=None)

# init the trainer and begin
trainer = Trainer(
    TrainerArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)
trainer.fit()
"""
#print(code)
myFile = open("/content/drive/MyDrive/" + dataset_name + "/train_vits.py", 'w')

myFile.write(code)
myFile.close()
%ls -al /content/drive/MyDrive/$dataset_name/train_vits.py

-rw------- 1 root root 3007 Feb  9 17:12 /content/drive/MyDrive/jennbroda-dataset/train_vits.py


Fine Tune VITS model

In [ ]:
!CUDA_VISIBLE_DEVICES="0" python /content/drive/MyDrive/$dataset_name/train_vits.py --restore_path /root/.local/share/tts/tts_models--en--ljspeech--vits/model_file.pth --config_file /root/.local/share/tts/tts_models--en--ljspeech--vits/config.json

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 | > Found 548 files in /content/drive/MyDrive/jennbroda-dataset
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent u

**Resume a fine tuning session**

Run the next cell to list all of the saved run direcories.  Copy select the run you want to resume, ctrl+C to copy the name. Paste in the next cell and run it.

In [ ]:
!ls /content/drive/MyDrive/$dataset_name/$output_directory

phoneme_cache


In [ ]:
run_name = "" #@param {type:"string"}

Run this cell to list all of the checkpoints in the saved run, then put the checkpoint name in the next cell, and run it.

In [ ]:
!ls /content/drive/MyDrive/$dataset_name/$output_directory/$run_name

phoneme_cache


In [ ]:
model_checkpoint = "best_model_1000011.pth" #@param {type:"string"}

Run the next cell to restore a previous fine tuning session using your dataset name, trainer output directory, and model checkpoint set

In [ ]:
!CUDA_VISIBLE_DEVICES="0" python /content/drive/MyDrive/$dataset_name/train_vits.py --restore_path /content/drive/MyDrive/$dataset_name/$output_directory/$run_name/$model_checkpoint --config_path /content/drive/MyDrive/$dataset_name/$run_name/config.json

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 | > Found 548 files in /content/drive/MyDrive/jennbroda-dataset
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent u

View Memory Usage

In [ ]:
!nvidia-smi

Thu Feb  9 17:22:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    33W /  70W |   6182MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Load Tensorboard

In [ ]:
import torch
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Load Dashboard. May take several minutes to appear from a blank white box.  Ad blockers probably need to whitelist a bunch of Colab stuff or this won't wrok.

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/$dataset_name/$output_directory/

Reusing TensorBoard on port 6006 (pid 26064), started 0:35:15 ago. (Use '!kill 26064' to kill it.)

<IPython.core.display.Javascript object>